In [ ]:
%cd ..
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_datasets_bw as datasets
from tensorflow_datasets_bw import visualize
import dppp

# Load an example dataset and example kernels

In [ ]:
scale_factor = 4
resize_method = tf.image.ResizeMethod.BICUBIC

builder_kwargs = {
    'resize_method': resize_method,
    'scale': scale_factor,
    'antialias': True
}
images = tfds.load(name="set14", split="test", builder_kwargs=builder_kwargs) \
                .map(datasets.map_on_dict(datasets.to_float32)) \
                .map(datasets.map_on_dict(datasets.from_255_to_1_range))

In [ ]:
image_dict = datasets.get_one_example(images, index=2)
image_hr = image_dict['hr'][None, ...]
image_lr_bicubic = image_dict['lr'][None, ...]

# Reconstruct the image - Bicubic

In [ ]:
# Load the denoiser
denoiser, (denoiser_min, denoiser_max) = dppp.load_denoiser('models/drugan+_0.0-0.2.h5')

# Callbacks: Print the PSNR every 5th step
callbacks = [ dppp.callback_print_psnr('psnr', 5, image_hr) ]

resize_fn = dppp.create_resize_fn(resize_method)
reconstructed_bicubic = dppp.hqs_super_resolve(image_lr_bicubic,
                                       scale_factor,
                                       denoiser,
                                       denoiser_max,
                                       resize_fn=resize_fn,
                                       callbacks=callbacks)

In [ ]:
# Print PSNR and FSIM
psnr = dppp.psnr(image_hr, reconstructed_bicubic).numpy()[0]
fsim = dppp.fsim(image_hr, reconstructed_bicubic).numpy()[0]
print(f"Reconstructed PSNR: {psnr:0.2f}, FSIM: {fsim:0.4f}")

# Visualize
visualize.draw_images([image_hr[0], image_lr_bicubic[0], reconstructed_bicubic[0]])

# Reconstruct the image - Gaussian (or any blur kernel)

## Create the downscaled image

In [ ]:
import math

# Create gaussian kernel
sigma = 2 * scale_factor / 6.0
size = math.ceil(sigma * 3)
kernel = dppp.conv2D_filter_rgb(dppp.gaussian_kernel(size, 0, sigma))

# Blur & Downsample
image_blurred = dppp.blur(image_hr, kernel, 0, mode='wrap')
image_lr_gaussian = image_blurred[:,::scale_factor,::scale_factor,:]

## Reconstruct

In [ ]:
# Load the denoiser
denoiser, (denoiser_min, denoiser_max) = dppp.load_denoiser('models/drugan+_0.0-0.2.h5')

# Callbacks: Print the PSNR every 5th step
callbacks = [ dppp.callback_print_psnr('psnr', 5, image_hr) ]

reconstructed_gaussian = dppp.hqs_super_resolve(image_lr_gaussian,
                                       scale_factor,
                                       denoiser,
                                       denoiser_max,
                                       kernel=kernel,
                                       callbacks=callbacks)

In [ ]:
# Print PSNR and FSIM
psnr = dppp.psnr(image_hr, reconstructed_gaussian).numpy()[0]
fsim = dppp.fsim(image_hr, reconstructed_gaussian).numpy()[0]
print(f"Reconstructed PSNR: {psnr:0.2f}, FSIM: {fsim:0.4f}")

# Visualize
visualize.draw_images([image_hr[0], image_lr_gaussian[0], reconstructed_gaussian[0]])